In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !unzip /content/drive/MyDrive/ClimaMM/climamm.zip -d /content

In [3]:
# !pip install torchnlp

In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
# from torchnlp import encoders
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import  mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from pathlib import Path
from tqdm.notebook import tqdm

import random

from transformers import AutoTokenizer, CLIPProcessor, CLIPModel

In [5]:
# Training settings
batch_size = 16
n_epochs = 7
patience = 25
lr=0.001
gamma = 0.7
weight_decay = 1e-5

seed = 510

# task= 'hate'
task= 'extremism'

model_used = 'momenta'

# same for both aspects(binary classification)
output_size = 1

device= 'cuda'

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

## Clip Setup

In [7]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=1a0418fe379b7b8e3958f5aa34f2a6af9932ebf3bf9ab96c509aa64d2c49996f
  Stored in directory: /root/.cache/pip/wheels/0d/17/90/042948fd2e2a87f1dcf6db6d438cad015c49db0c53d1d9c7dc
Successfully built openai-clip


In [8]:
import clip

# clip_model = torch.jit.load("clip_model.pt").cuda().eval()

# clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").cuda().eval()

clip_model, preprocess = clip.load("ViT-B/32", device=device)

# print(clip_model)

# input_resolution = clip_model.input_resolution.item()

# context_length = clip_model.context_length.item()
# vocab_size = clip_model.vocab_size.item()

input_resolution = 224
context_length = 77
vocab_size = 49408

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 69.2MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


## Image Setup

In [9]:
preprocess = Compose([
    Resize(input_resolution, interpolation=Image.BICUBIC),
    CenterCrop(input_resolution),
    ToTensor()
    ])

# Get the image features for a single image input
def process_image_clip(in_img):
    image_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
    image_std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()

    image = preprocess(Image.open(in_img).convert("RGB"))

    image_input = torch.tensor(np.stack(image)).cuda()
    image_input -= image_mean[:, None, None]
    image_input /= image_std[:, None, None]
    return image_input

## Text Setup

In [10]:
import gzip
import html
import os
from functools import lru_cache

import ftfy
import regex as re


@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def basic_clean(text):
    text = ftfy.fix_text(text)
    text = html.unescape(html.unescape(text))
    return text.strip()


def whitespace_clean(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text


class SimpleTokenizer(object):
    def __init__(self, bpe_path: str = "bpe_simple_vocab_16e6.txt.gz"):
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        merges = gzip.open(bpe_path).read().decode("utf-8").split('\n')
        merges = merges[1:49152-256-2+1]
        merges = [tuple(merge.split()) for merge in merges]
        vocab = list(bytes_to_unicode().values())
        vocab = vocab + [v+'</w>' for v in vocab]
        for merge in merges:
            vocab.append(''.join(merge))
        vocab.extend(['<|startoftext|>', '<|endoftext|>'])
        self.encoder = dict(zip(vocab, range(len(vocab))))
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.bpe_ranks = dict(zip(merges, range(len(merges))))
        self.cache = {'<|startoftext|>': '<|startoftext|>', '<|endoftext|>': '<|endoftext|>'}
        self.pat = re.compile(r"""<\|startoftext\|>|<\|endoftext\|>|'s|'t|'re|'ve|'m|'ll|'d|[\p{L}]+|[\p{N}]|[^\s\p{L}\p{N}]+""", re.IGNORECASE)

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token[:-1]) + ( token[-1] + '</w>',)
        pairs = get_pairs(word)

        if not pairs:
            return token+'</w>'

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        text = whitespace_clean(basic_clean(text)).lower()
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors="replace").replace('</w>', ' ')
        return text


In [11]:
# tokenizer = SimpleTokenizer()

# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")
# text_features = model.get_text_features(**inputs)


# Get the text features for a single text input
def process_text_clip(in_text):
    text_token = tokenizer.encode(in_text)
    text_input = torch.zeros(context_length, dtype=torch.long)
    tokens = text_token[:77]
    text_input[:len(tokens)] = torch.tensor(tokens)
    text_input = text_input.cuda()
    return text_input

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## Dataset Setup

In [12]:
class ImageDataset(Dataset):
    def __init__(self, data, img_dir, split, transform=None):
        self.data = data[data['split'] == split].reset_index(drop=True)
        if task == 'target':
            self.data = self.data[self.data['hate'] == 1].reset_index(drop=True)

        if task == 'extremism':
            self.data = self.data[self.data['stance'].isin([1, 2])].reset_index(drop=True)

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = self.data.loc[idx, 'name']
        img_name = os.path.join(self.img_dir, self.data.loc[idx, 'name'])
        text = self.data.loc[idx, 'text']

        image_clip_input = process_image_clip(img_name)
        text_clip_input = process_text_clip(text)

        # image = Image.open(img_name).convert('RGB')

        lab = self.data.loc[idx, task]
        label = torch.tensor(lab).to(device)

        sample = {
                "id": img_id,
                "image_clip_input": image_clip_input,
                "text_clip_input": text_clip_input,
                "label": label
            }

        # if self.transform:
        #   image = self.transform(image)

        return sample

In [13]:
import os
import pandas as pd
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ─── 1) Hyperparameters & Paths ────────────────────────────────────────────────
batch_size    = 32
num_workers   = 2

TRAIN_IMG_DIR = "/content/drive/MyDrive/SharedTaskProject/Subtask_A/Train/Subtask_A_Train"
EVAL_CSV      = "/content/drive/MyDrive/SharedTaskProject/Eval_Labels/STask-A(index,label)val.csv"
EVAL_IMG_DIR  = "/content/drive/MyDrive/SharedTaskProject/Subtask_A/Evaluation"


# ─── 2) Preprocessing ──────────────────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),

])

# ─── 3) Training Dataset & Loader ─────────────────────────────────────────────
# ImageFolder will assign class indices based on subfolder names: 'Hate'→0, 'NoHate'→1 (alphabetical)
train_dataset   = ImageFolder(root=TRAIN_IMG_DIR, transform=preprocess)
dataloader_train = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

# ─── 4) Dev/Eval Dataset & Loader ──────────────────────────────────────────────
class EvalDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform):
        self.df        = pd.read_csv(csv_path)
        self.img_dir   = img_dir
        self.transform = transform
        self.label_map = {'NoHate': 0, 'Hate': 1}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row   = self.df.iloc[idx]
        img_p = os.path.join(self.img_dir, row['index'])
        image = Image.open(img_p).convert("RGB")
        image = self.transform(image)
        label = self.label_map[row['HateLabel']]
        return image, label

eval_dataset   = EvalDataset(EVAL_CSV, EVAL_IMG_DIR, transform=preprocess)
dataloader_val = DataLoader(
    eval_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)


# ─── Summary ────────────────────────────────────────────────────────────────────
print(f"TRAIN 🚀  samples: {len(train_dataset)}")
print(f"EVAL 🔍   samples: {len(eval_dataset)}")



TRAIN 🚀  samples: 4050
EVAL 🔍   samples: 506


In [14]:
print(len(dataloader_train.dataset))
print(len(dataloader_val.dataset))
# print(len(dataloader_test.dataset))

4050
506


## Model Setup

In [15]:
class MM(nn.Module):
    def __init__(self, n_out):
        super(MM, self).__init__()

        self.dense_vgg_1024 = nn.Linear(4096, 1024)
        self.dense_vgg_512 = nn.Linear(1024, 512)
        self.drop20 = nn.Dropout(p=0.2)
        self.drop5 = nn.Dropout(p=0.05)

        self.dense_drob_512 = nn.Linear(768, 512)

        self.gen_key_L1 = nn.Linear(512, 256) # 512X256
        self.gen_query_L1 = nn.Linear(512, 256) # 512X256
        self.gen_key_L2 = nn.Linear(512, 256) # 512X256
        self.gen_query_L2 = nn.Linear(512, 256) # 512X256
        self.gen_key_L3 = nn.Linear(512, 256) # 512X256
        self.gen_query_L3 = nn.Linear(512, 256) # 512X256
#         self.gen_value = nn.Linear(512, 256) # 512X256
        self.soft = nn.Softmax(dim=1)
        self.soft_final = nn.Softmax(dim=1)
        self.project_dense_512a = nn.Linear(1024, 512) # 512X256
        self.project_dense_512b = nn.Linear(1024, 512) # 512X256
        self.project_dense_512c = nn.Linear(1024, 512) # 512X256


        self.fc_out = nn.Linear(512, 256) # 512X256
        self.out = nn.Linear(256, n_out) # 512X256


    def selfattNFuse_L1a(self, vec1, vec2):
            q1 = F.relu(self.gen_query_L1(vec1))
            k1 = F.relu(self.gen_key_L1(vec1))
            q2 = F.relu(self.gen_query_L1(vec2))
            k2 = F.relu(self.gen_key_L1(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512a(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep

    def selfattNFuse_L1b(self, vec1, vec2):
            q1 = F.relu(self.gen_query_L2(vec1))
            k1 = F.relu(self.gen_key_L2(vec1))
            q2 = F.relu(self.gen_query_L2(vec2))
            k2 = F.relu(self.gen_key_L2(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512b(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep

    def selfattNFuse_L2(self, vec1, vec2):
            q1 = F.relu(self.gen_query_L3(vec1))
            k1 = F.relu(self.gen_key_L3(vec1))
            q2 = F.relu(self.gen_query_L3(vec2))
            k2 = F.relu(self.gen_key_L3(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512c(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep

    def forward(self, in_CI, in_CT):
        # VGG_feat = self.drop20(F.relu(self.dense_vgg_512(self.drop20(F.relu(self.dense_vgg_1024(in_VGG))))))
        # Drob_feat = self.drop5(F.relu(self.dense_drob_512(in_Drob)))
        # out_img = self.selfattNFuse_L1a(VGG_feat, in_CI)
        # out_txt = self.selfattNFuse_L1b(Drob_feat, in_CT)
        # out_img_txt = self.selfattNFuse_L2(out_img, out_txt)

        out_img_txt = self.selfattNFuse_L2(in_CI, in_CT)

        final_out = F.relu(self.fc_out(out_img_txt))

        out = torch.sigmoid(self.out(final_out)) #For binary case
        # out = self.out(final_out)
        return out

In [16]:

exp_name = "EMNLP_MCHarm_GLAREAll_COVTrain_POLEval"
# pre_trn_ckp = "EMNLP_MCHarm_GLAREAll_COVTrain" # Uncomment for using pre-trained
exp_path = "path_to_saved_files/EMNLP_ModelCkpt/"+exp_name

criterion = nn.BCELoss() #Binary case

# criterion = nn.CrossEntropyLoss() #Multiclass case


model = MM(output_size)
model.to(device)
# print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

## Train Setup

In [17]:
!pip install pytorchtools

### Binary Train

In [18]:
#For binary

def train_model(model, patience, n_epochs):
    epochs = n_epochs
#     clip = 5

    train_acc_list=[]
    val_acc_list=[]
    train_loss_list=[]
    val_loss_list=[]

    # initialize the experiment path
    Path(exp_path).mkdir(parents=True, exist_ok=True)
    # initialize early_stopping object
    # chk_file = os.path.join(exp_path, 'checkpoint_'+exp_name+'.pt')
    # early_stopping = EarlyStopping(patience=patience, verbose=True, path=chk_file)

    model.train()
    for i in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        for data in tqdm(dataloader_train):

#             Clip features...
            # Access data using string keys for dataloader_train as ImageDataset returns a dictionary
            img_inp_clip = data['image_clip_input']
            txt_inp_clip = data['text_clip_input']
            with torch.no_grad():
                img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

            # img_feat_vgg = data['image_vgg_feature']
#             txt_feat_trans = data['text_drob_embedding']

            label = data['label'].to(device)

            optimizer.zero_grad() # Use optimizer.zero_grad() instead of model.zero_grad()
            # output = model(img_feat_clip, img_feat_vgg, txt_feat_clip)

            output = model(img_feat_clip, txt_feat_clip)

#             output = model(img_feat_vgg, txt_feat_trans)

            loss = criterion(output.squeeze(), label.float())

#             print(loss)
            loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            with torch.no_grad():
#                 print(output.squeeze().shape)
#                 print(label.float().shape)
                # Calculate accuracy for binary classification
                predicted = (output.squeeze() > 0.5).float()
                acc = (predicted == label.float()).sum().item() / label.size(0)
                total_acc_train += acc
                total_loss_train += loss.item()

        train_acc = total_acc_train/len(dataloader_train)
        train_loss = total_loss_train/len(dataloader_train)
        model.eval()
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for data in tqdm(dataloader_val):
#                 Clip features...
                # Access data using integer indices for dataloader_val as EvalDataset returns a tuple
                img_inp_clip = data[0].to(device) # Image tensor is at index 0
                txt_inp_clip = data[1].to(device) # Text tensor is at index 1 (assuming EvalDataset is modified to return text)
                label = data[2].to(device) # Label tensor is at index 2 (assuming EvalDataset is modified to return text)

                with torch.no_grad():
                    img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                    txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)


                # img_feat_vgg = data['image_vgg_feature']
#                 txt_feat_trans = data['text_drob_embedding']

                # output = model(img_feat_clip, img_feat_vgg, txt_feat_clip)
                output = model(img_feat_clip, txt_feat_clip)

#                 output = model(img_feat_vgg, txt_feat_trans)


                val_loss = criterion(output.squeeze(), label.float())
                 # Calculate accuracy for binary classification
                predicted = (output.squeeze() > 0.5).float()
                acc = (predicted == label.float()).sum().item() / label.size(0)
                total_acc_val += acc
                total_loss_val += val_loss.item()

        print("Saving model...")
        torch.save(model.state_dict(), os.path.join(exp_path, "final.pt"))

        val_acc = total_acc_val/len(dataloader_val)
        val_loss = total_loss_val/len(dataloader_val)

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)

        # early_stopping(val_loss, model)

        # if early_stopping.early_stop:
        #     print("Early stopping")
        #     break

        print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
        model.train()
        torch.cuda.empty_cache()

    # load the last checkpoint with the best model
#     model.load_state_dict(torch.load(chk_file))

    return  model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, i

### Multiclass train

In [19]:


# def train_model(model, patience, n_epochs):
#     epochs = n_epochs

#     train_acc_list=[]
#     val_acc_list=[]
#     train_loss_list=[]
#     val_loss_list=[]

#         # initialize the experiment path
#     Path(exp_path).mkdir(parents=True, exist_ok=True)


#     model.train()
#     for i in range(epochs):
# #         total_acc_train = 0
#         total_loss_train = 0
#         total_train = 0
#         correct_train = 0

#         for data in tqdm(dataloader_train):

# #             Clip features...
#             img_inp_clip = data['image_clip_input']
#             txt_inp_clip = data['text_clip_input']
#             with torch.no_grad():
#                 img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
#                 txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

#             # img_feat_vgg = data['image_vgg_feature']
#             # txt_feat_trans = data['text_drob_embedding']

#             label_train = data['label'].type(torch.LongTensor)

#             label_train = label_train.to(device)

#             model.zero_grad()

#             output = model(img_feat_clip, txt_feat_clip)

#             # output = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)

#           #  print(output.shape)


#             loss = criterion(output.squeeze(), label_train)

# #             print(loss)
#             loss.backward()
# #             nn.utils.clip_grad_norm_(model.parameters(), clip)
#             optimizer.step()

#             with torch.no_grad():
#                 _, predicted_train = torch.max(output.data, 1)
#                 total_train += label_train.size(0)
#                 correct_train += (predicted_train == label_train).sum().item()
# #                 out_val = (output.squeeze()>0.5).float()
# #                 out_final = ((out_val == 1).nonzero(as_tuple=True)[0])
# #                 print()
# #                 acc = torch.abs(output.squeeze() - label.float()).view(-1)
# #                 acc = (1. - acc.sum() / acc.size()[0])
# #                 total_acc_train += acc
#                 total_loss_train += loss.item()


#         train_acc = 100 * correct_train / total_train
#         train_loss = total_loss_train/total_train
#         model.eval()
# #         total_acc_val = 0
#         total_loss_val = 0
#         total_val = 0
#         correct_val = 0

#         with torch.no_grad():
#             for data in tqdm(dataloader_val):
# #                 Clip features...
#                 img_inp_clip = data['image_clip_input']
#                 txt_inp_clip = data['text_clip_input']
#                 with torch.no_grad():
#                     img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
#                     txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)


#                 # img_feat_vgg = data['image_vgg_feature']
#                 # txt_feat_trans = data['text_drob_embedding']



#                 label_val = data['label'].type(torch.LongTensor)

#                 label_val = label_val.to(device)

#                 model.zero_grad()

#                 output = model(img_feat_clip, txt_feat_clip)

#                 # output = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)


#                 val_loss = criterion(output.squeeze(), label_val)
#                 _, predicted_val = torch.max(output.data, 1)
#                 total_val += label_val.size(0)
#                 correct_val += (predicted_val == label_val).sum().item()
#                 total_loss_val += val_loss.item()
#         print("Saving model...")
#         torch.save(model.state_dict(), os.path.join(exp_path, "final.pt"))

#         val_acc = 100 * correct_val / total_val
#         val_loss = total_loss_val/total_val

#         train_acc_list.append(train_acc)
#         val_acc_list.append(val_acc)
#         train_loss_list.append(train_loss)
#         val_loss_list.append(val_loss)

#         # early_stopping(val_loss, model)

#         # if early_stopping.early_stop:
#         #     print("Early stopping")
#         #     break

#         print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
#         model.train()
#         torch.cuda.empty_cache()

#     # load the last checkpoint with the best model
# #     model.load_state_dict(torch.load('checkpoint_1.pt'))

#     return  model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, i


## Train

In [20]:
model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, epoc_num = train_model(model, patience, n_epochs)


  0%|          | 0/127 [00:00<?, ?it/s]

TypeError: list indices must be integers or slices, not str

## Test and Eval Setup

In [ ]:
!pip install torchmetrics

import torchmetrics

### Binary Test

In [ ]:

auroc = torchmetrics.AUROC(task = 'binary', num_classes = output_size)

# For BCE loss
def test_model(model):
    model.eval()
    total_acc_test = 0
    total_loss_test = 0
    outputs = []
    test_labels=[]
    with torch.no_grad():
        for data in tqdm(dataloader_test):
            img_inp_clip = data['image_clip_input']
            txt_inp_clip = data['text_clip_input']
            with torch.no_grad():
                img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

            # img_feat_vgg = data['image_vgg_feature']
#             txt_feat_trans = data['text_drob_embedding']

            label = data['label'].to(device)

#             out = model(img_feat_vgg, txt_feat_trans)

            # out = model(img_feat_clip, img_feat_vgg, txt_feat_clip)

            out = model(img_feat_clip, txt_feat_clip)

            outputs += list(out.cpu().data.numpy())



            preds_proxy = torch.sigmoid(out)

            auroc.update(preds_proxy, data['label'])



            loss = criterion(out.squeeze(), label.float())
#             print(out.squeeze())
#             print(label.float())
            acc = torch.abs(out.squeeze() - label.float()).view(-1)
    #         print((acc.sum() / acc.size()[0]))
            acc = (1. - acc.sum() / acc.size()[0])
    #         print(acc)
            total_acc_test += acc
            total_loss_test += loss.item()

    acc_test = total_acc_test/len(dataloader_test)
    loss_test = total_loss_test/len(dataloader_test)
    print(f'acc: {acc_test:.4f} loss: {loss_test:.4f}')
    return outputs


### Multiclass Test

In [ ]:
# auroc = torchmetrics.AUROC(task = 'multiclass', num_classes = output_size)

# #Multiclass
# def test_model(model):
#     model.eval()
#     total_test = 0
#     correct_test =0
#     total_acc_test = 0
#     total_loss_test = 0
#     outputs = []
#     test_labels=[]
#     with torch.no_grad():
#         for data in tqdm(dataloader_test):
#             img_inp_clip = data['image_clip_input']
#             txt_inp_clip = data['text_clip_input']
#             with torch.no_grad():
#                 img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
#                 txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

#             # img_feat_vgg = data['image_vgg_feature']
#             # txt_feat_trans = data['text_drob_embedding']

#             label_test = data['label'].type(torch.LongTensor)
#             label_test = label_test.to(device)

# #             out = model(img_feat_vgg, txt_feat_trans)

#             output = model(img_feat_clip, txt_feat_clip)

#             # out = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)

#             outputs += list(output.cpu().data.numpy())
#             loss = criterion(output.squeeze(), label_test)

#             _, predicted_test = torch.max(output.data, 1)
#             total_test += label_test.size(0)
#             correct_test += (predicted_test == label_test).sum().item()
#             total_loss_test += loss.item()


#             preds_proxy = torch.sigmoid(output)

#             auroc.update(preds_proxy, label_test)


#     acc_test = 100 * correct_test / total_test
#     loss_test = total_loss_test/total_test

#     print(f'acc: {acc_test:.4f} loss: {loss_test:.4f}')
#     return outputs

## Testing

In [ ]:
outputs = test_model(model)

# Binary setting
np_out = np.array(outputs)
y_pred = np.zeros(np_out.shape)
y_pred[np_out>0.5]=1
y_pred = np.array(y_pred)


# # Multiclass setting
# y_pred=[]
# for i in outputs:
#     y_pred.append(np.argmax(i))
# # # np.argmax(outputs[:])
# # outputs



test_labels=[]


for data in dataloader_test:
    label_test = data['label'].to(device)
    l2 = label_test.cpu().detach().numpy()
    test_labels.extend(l2)

In [ ]:

print("Model: ", model_used )
print("Seed: ", seed )
print("Task: ", task )

print("Accuracy:{:.4f}".format(accuracy_score(test_labels, y_pred) ))
print("Recall:{:.4f}".format(recall_score(test_labels, y_pred,average='macro') ))
print("Precision:{:.4f}".format(precision_score(test_labels, y_pred,average='macro') ))
print("f1_score:{:.4f}".format(f1_score(test_labels, y_pred,average='macro') ))

print("AUC:{:.4f}".format(auroc.compute()))

print(classification_report(test_labels,y_pred))
print(confusion_matrix(y_pred,test_labels))
